<a href="https://colab.research.google.com/github/visheshagrawal/IR2022_A1_96/blob/main/IR_A1_Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**IR Assignment 1** :: **Question 2**

Team Members:

Aditi Soni (2018326)

Vishesh Agrawal (2018420)


#Importing the Libraries

In [ ]:
# Importing the Libraries

from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer,SnowballStemmer,WordNetLemmatizer
import json
import os
from tqdm.notebook import tqdm,tnrange
import string
import numpy as np
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from collections import defaultdict
from tabulate import tabulate
from functools import reduce
import copy

nltk.download("punkt")
nltk.download("stopwords")

#Converting the Documents into positional indexes 

In [ ]:
%cd /content/drive/MyDrive/IR/HW1/Dataset/
%ls

/content/drive/.shortcut-targets-by-id/15TiqBLyk8RJBhmc1guzZPOx9pU0Ch0f6/IR/HW1/Dataset


In [67]:
old_db={}
def pre_processing(text):
  stopWords = set(stopwords.words('english'))
  #convert text into lowercase to uniform it
  lower_text = text.lower()
  #Tokenization using word tokenize                                     
  tokens = word_tokenize(lower_text)                       
  #removing stopwords    
  tokens_wo_stopwords = []
  for token in tokens:
    if token not in stopWords:
      tokens_wo_stopwords.append(token)
  #removing punctuation marks
  tokens_wo_stopwords_punct = []
  for x in tokens_wo_stopwords:
    tokens_wo_stopwords_punct.append(remove_special_char(x))   
  return tokens_wo_stopwords_punct


def index_file(file,fileId):
  tokens = pre_processing(file)
  for pos,val in enumerate(tokens):
    if val in old_db:
      old_db[val][0]+=1
      if(fileId in old_db[val][1]):
        old_db[val][1][fileId].append(pos)
      else:
        old_db[val][1][fileId]=[pos]  
    else:
      old_db[val] = [1,{fileId:[pos]}]

def remove_special_char(text):
  punct_char = string.punctuation
  new_string=''
  for char in text:
    if char.isalnum() and not char.isdigit():
      if char not in punct_char:
        new_string+=char
  return new_string

def file_save(dbf):
  json.dump(dbf, open('output_q3.json', "w"))

Open all the docs, first map their paths and second generate output.json which would have positional embeddings

In [68]:
files_list = os.listdir("Humor,Hist,Media,Food")
fpath=[]
count=0
for file in files_list:
    count += 1
    path2 = os.path.join("Humor,Hist,Media,Food", file)
    fpath.append(path2)

# json.dump(fpath, open("mapping_q2.json", "w"))
json.dump(fpath, open("mapping_q3.json", "w"))


for i,filePath in enumerate(tqdm(fpath)):
    try:
        file = open(filePath, encoding="utf8")
        read = file.read().replace('\n', ' ')    
    except Exception as e:
        file = open(filePath, encoding="unicode_escape")
        read = file.read().replace('\n', ' ')
    file.close()
    #Will automatically update db
    index_file(read, i)
    #Save and dump that db in a json file
    file_save(old_db)

  0%|          | 0/1133 [00:00<?, ?it/s]

#Query Part Handling

Following the **Preprocessing Format** of our Text


1.   Convert the text to lower case
2.   Perform word tokenization
1.   Remove stopwords from tokens
2.   Remove punctuation marks from tokens
1.   Remove blank space tokens








In [70]:
def remove_special_char(text):
  punct_char = string.punctuation
  new_string=''
  for char in text:
    if char.isalnum() and not char.isdigit():
      if char not in punct_char:
        new_string+=char
  return new_string

In [71]:
def get_postings(terms):
    ret_list=[]
    for term in terms:
      ret_list.append([])
      for i in db[term][1]:
        ret_list[-1].append([i,db[term][1][i]])
    return ret_list

def get_docs_from_postings(postings):
    ret_list=[]
    for p in postings:
      ret_list.append([])
      for x in p:
        ret_list[-1].append(x[0])
    return ret_list

In [72]:
def intersect_lists(lists):
  if len(lists)==0:
      return []
  lists.sort(key=len)
  ret_list=set(lists[0])
  for i in range(len(lists)-1):
    ret_list=set(ret_list)&set(lists[i+1])
  ret_list=list(ret_list)
  return ret_list

In [73]:
def pre_processing(text):
  stopWords = set(stopwords.words('english'))
  #convert text into lowercase to uniform it
  lower_text = text.lower()
  #Tokenization using word tokenize                                     
  tokens = word_tokenize(lower_text)                       
  #removing stopwords    
  tokens_wo_stopwords = []
  for token in tokens:
    if token not in stopWords:
      tokens_wo_stopwords.append(token)
  #removing punctuation marks
  tokens_wo_stopwords_punct = []
  for x in tokens_wo_stopwords:
    tokens_wo_stopwords_punct.append(remove_special_char(x))   
  #Removing blank space tokens
  validTokens = []
  for x in tokens_wo_stopwords_punct:
    if len(x)>1:
      validTokens.append(x)
  print(validTokens)
  return validTokens

In [74]:
def get_mapping(files):
  print('Total Documents:',len(files))
  mapping=json.load(open('mapping_q3.json'))
  data= list(map(lambda i:(i,mapping[i]),files)) 
  data.sort()
  print(tabulate(data,headers=['Document ID','Location']))

In [75]:
#Load the output file and use db
db=json.load(open('output_q3.json'))
db=defaultdict(lambda:[],db)

In [76]:
def perform_query(phrase):
  phrase_pp= pre_processing(phrase)    
  print("Query:",phrase_pp)
  
  # if(len(db.keys())==0):      
  #     self.load()
  
  for term in phrase_pp:               
    if(term not in db):
      return []

  if(len(phrase_pp)==1):                      
    result= db[phrase_pp[0]][1].keys()
    result=list(result)

  else:
    postings=get_postings(phrase_pp)    
    docs=get_docs_from_postings(postings)
    docs=intersect_lists(docs)     

    for i in range(len(postings)):      
      postings[i]=[x for x in postings[i] if x[0] in docs]
    
    postings=copy.deepcopy(postings)
  
    for i in range(len(postings)):      
      for j in range(len(postings[i])):
        postings[i][j][1]=[x-i for x in postings[i][j][1]]

    
    result=[]
    for i in range(len(postings[0])):
      intersection=intersect_lists([x[i][1] for x in postings]) 
      if len(intersection)>0:
        result.append(postings[0][i][0])            

  result=list(map(int,result))    
  get_mapping(result)         
  return result

In [79]:
# phrase = input()
# phrase = "real aphrodisiacs obviously"
phrase= "has long been known that"
output = perform_query(phrase)

['long', 'known']
Query: ['long', 'known']
Total Documents: 7
  Document ID  Location
-------------  -----------------------------------
           52  Humor,Hist,Media,Food/llong.hum
          150  Humor,Hist,Media,Food/research.hum
          366  Humor,Hist,Media,Food/jargon.phd
          368  Humor,Hist,Media,Food/lampoon.jok
          409  Humor,Hist,Media,Food/resrch_phrase
          923  Humor,Hist,Media,Food/acne1.txt
         1123  Humor,Hist,Media,Food/resrch_p.hra


In [ ]:
# file = open("Humor,Hist,Media,Food/aphrodis.txt")
# file.read()

"HERBSAPHRODISIACSSEXHERBALISTNARCOTICDAMIANAAQUARIAN\nAPHRODISIACS - Aquarian\n\n\nAPHRODISIACS by Aquarian\n------------------------\n\nAphrodisiacs! The word itself can send a tingle of anticipation\ndown your spine. Throughout the ages almost every culture has\nused various `substances', usually herbal in origin, to put some\nzip into their love lives or in an attempt to cure the impotent.\nThe types of preparation employed as aphrodisiacs ranged from the\nuseless (except perhaps for their psychological effects) to the\nextremely dangerous, some being toxic enough to cause death!\nHowever, leaving those dangers behind....\n\nThere were four main types of herbal substances used as\naphrodisiacs. The first and most dangerous was the narcotic, used\nto intoxicate the user's object of desire, thereby rendering them\nincapable of responding to an unwanted sexual advance. These\naren't real aphrodisiacs obviously, just a way of weakening\nanother person's will. These substances are extre